In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")
llm.invoke("What is the capital of France?")
# llm.invoke(0)


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-07-30T06:14:28.089657Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1710580542, 'load_duration': 1283963375, 'prompt_eval_count': 32, 'prompt_eval_duration': 335386250, 'eval_count': 8, 'eval_duration': 89864584, 'model_name': 'llama3.2:1b'}, id='run--1966edca-2aaf-4d94-bf14-a4f51991c28f-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)

# PromptValue 타입
prompt = prompt_template.invoke({"country":"France"})

ai_message = llm.invoke(prompt)
print(ai_message)

output_parser = StrOutputParser()
answer = output_parser.invoke(llm.invoke(prompt))
print(answer)

content='Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-07-30T06:20:06.797157Z', 'done': True, 'done_reason': 'stop', 'total_duration': 443494917, 'load_duration': 38447334, 'prompt_eval_count': 39, 'prompt_eval_duration': 381937000, 'eval_count': 3, 'eval_duration': 22493625, 'model_name': 'llama3.2:1b'} id='run--25a3f38c-87bb-4afb-8b74-910499047165-0' usage_metadata={'input_tokens': 39, 'output_tokens': 3, 'total_tokens': 42}
Paris.


In [4]:
ai_message

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-07-30T06:18:45.822847Z', 'done': True, 'done_reason': 'stop', 'total_duration': 432508542, 'load_duration': 51824875, 'prompt_eval_count': 32, 'prompt_eval_duration': 303368667, 'eval_count': 8, 'eval_duration': 76719250, 'model_name': 'llama3.2:1b'}, id='run--de086c68-b7eb-4706-9a7e-6d9966ca80b8-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [ ]:
from pydantic import BaseModel, Field

# json 형식을 응답받기를 원한다면 사용
class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

structured_llm = llm.with_structured_output(CountryDetail)

In [28]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    return it is JSON format, and return the JSON dictionary only
    """,
    input_variables=["country"],
)

prompt = country_detail_prompt.invoke({"country": "France"})

# llm.invoke 결과가 str이고, 결과 포맷이 그때그때 달라서 사용할 수 없음(간헐적으로 에러 발생)
# output_parser = JsonOutputParser()
# json_ai_message = llm.invoke(country_detail_prompt.invoke({"country": "France"}))
# output_parser.invoke(json_ai_message)

json_ai_message = structured_llm.invoke(country_detail_prompt.invoke({"country": "France"}))

In [32]:
json_ai_message
# str 타입임
# type(json_ai_message.content)

# json
json_ai_message.model_dump()['capital']

'Paris'